# LSP-00-05 Test Case

This performs the test **LSP-00-05: Demonstration of low-volume and/or indexed queries against the WISE data via API**.

In [1]:
import time
import requests

In [2]:
import xml.etree.ElementTree as ET

In [3]:
import numpy as np

## Service definitions

In [4]:
daxUrl = 'http://lsst-qserv-dax01.ncsa.illinois.edu:5000/'

In [5]:
tapUrl = daxUrl + 'db/v0/tap/sync'

In [6]:
ipacTapUrl = 'https://irsa.ipac.caltech.edu/TAP/sync'

## Test case configuration

As defined in LDM-540, the tables relevant to this test case are wise_00.allwise_p3as_psd (similar to LSST Object - AllWISE coadded Source Catalog), wise_00.allwise_p3as_mep (similar to LSST Forced Source - AllWISE Multiepoch Photometry Table), and two of the four Single-Exposure Source Databases (similar to LSST Source): wise_4band_00.allsky_4band_p1bs_psd (original mission, 4-band cold phase), and neowiser_yr1_00.neowiser_yr1_p1bs_psd (first year of reactivated mission).  The two smaller, transitional, 3-band and 2-band tables are not queried.

In [7]:
caseTables = ['wise_00.allwise_p3as_psd',
              'wise_00.allwise_p3as_mep',
              'wise_4band_00.allsky_4band_p1bs_psd',
              'neowiser_yr1_00.neowiser_yr1_p1bs_psd']
casePairs = [ ( pdacT, ( pdacT.split('.')[1] if 'neowise' not in pdacT else 'neowiser_p1bs_psd' ) ) for pdacT in caseTables ]
casePairs

[('wise_00.allwise_p3as_psd', 'allwise_p3as_psd'),
 ('wise_00.allwise_p3as_mep', 'allwise_p3as_mep'),
 ('wise_4band_00.allsky_4band_p1bs_psd', 'allsky_4band_p1bs_psd'),
 ('neowiser_yr1_00.neowiser_yr1_p1bs_psd', 'neowiser_p1bs_psd')]

## Query Functions

Set up functions for sending queries to PDAC, for JSON output, and IPAC, for VOTable XML output.

In [8]:
def doPDACQuery( qString, timing=True ):
    print( 'PDAC: Executing "' + qString + '"' )
    query = { 'query' : qString }
    if timing:
        befT = time.time()
        print( 'Before query (wall,cpu):', befT, time.process_time() )
    result = requests.post(tapUrl,data=query)
    if timing:
        aftT = time.time()
        print( 'After  query (wall,cpu,elapsed):', aftT, time.process_time(), aftT-befT )
    print( 'PDAC query status code:', result.status_code )
    return result.json()

In [9]:
def doIPACQuery( qString, timing=True ):
    print( 'PDAC: Executing "' + qString + '"' )
    if timing:
        befT = time.time()
        print( 'Before query (wall,cpu):', befT, time.process_time() )
    result = requests.get( ipacTapUrl + '?QUERY=' + qString )
    if timing:
        aftT = time.time()
        print( 'After  query (wall,cpu,elapsed):', aftT, time.process_time(), aftT-befT )
    print( 'IPAC query status code:', result.status_code )
    return result.text

## Sample Queries

### Cone searches

We begin with test searches for a small cone around a location in the Galactic plane, from both PDAC and the production WISE TAP service.

In [10]:
testCoords = ( 281.5, -2.6 ) # semi-random location in the Galactic plane, near (30,0)
testRadius = 100/3600        # 100 arcseconds is maximum cone search radius in Portal for now

The PDAC test uses a Qserv-specific query function in SQL.  It's not really ADQL at all.  The actual position arguments (ra, decl) from the table being queried are _implicit_ arguments.

In [11]:
qservWhere = 'WHERE qserv_areaspec_circle( ' \
+ repr(testCoords[0]) + ', ' + repr(testCoords[1]) + ', ' + repr(testRadius) + ' )'
qservWhere

'WHERE qserv_areaspec_circle( 281.5, -2.6, 0.027777777777777776 )'

The IPAC TAP query uses simple ADQL 2.0.  Note that it appears that this `CONTAINS(POINT,CIRCLE)` idiom will be deprecated from ADQL 2.1 on.

In [12]:
adqlWhere = 'WHERE CONTAINS(POINT(\'J2000\',ra,dec),CIRCLE(\'J2000\',' \
+ repr(testCoords[0]) + ',' + repr(testCoords[1]) + ',' + repr(testRadius) + '))=1'
adqlWhere

"WHERE CONTAINS(POINT('J2000',ra,dec),CIRCLE('J2000',281.5,-2.6,0.027777777777777776))=1"

#### AllWISE Source Catalog, i.e., "Object"

First we query the coadded source catalog, most similar to the planned LSST "Object" table.  We select the same data from the tables on both sites.

In [13]:
wiseSelect = 'SELECT source_id, ra, decl, w1mpro, w2mpro'

In [14]:
wisePDACQ = doPDACQuery( wiseSelect + ' FROM wise_00.allwise_p3as_psd ' 
                        + qservWhere )

PDAC: Executing "SELECT source_id, ra, decl, w1mpro, w2mpro FROM wise_00.allwise_p3as_psd WHERE qserv_areaspec_circle( 281.5, -2.6, 0.027777777777777776 )"
Before query (wall,cpu): 1528229878.934869 1.929485
After  query (wall,cpu,elapsed): 1528229879.160303 1.954729 0.2254340648651123
PDAC query status code: 200


In [15]:
wisePDACQ

{'result': {'table': {'metadata': {'elements': [{'name': 'source_id',
      'datatype': 'text'},
     {'name': 'ra', 'datatype': 'double'},
     {'name': 'decl', 'datatype': 'double'},
     {'name': 'w1mpro', 'datatype': 'double'},
     {'name': 'w2mpro', 'datatype': 'double'}]},
   'data': [['2813m031_ac51-042175', 281.4952743, -2.5792752, 8.776, 7.861],
    ['2813m031_ac51-043072', 281.4764563, -2.5948072, 10.635, 10.834],
    ['2813m031_ac51-042944', 281.4845407, -2.5966385, 10.5, 10.443],
    ['2813m031_ac51-046306', 281.5217076, -2.6025986, 11.7, 12.45],
    ['2813m031_ac51-045379', 281.5163201, -2.6086848, 12.003, 10.788],
    ['2813m031_ac51-042882', 281.4907977, -2.5963688, 10.603, 10.667],
    ['2813m031_ac51-045039', 281.5105153, -2.5855097, 11.083, 10.425],
    ['2813m031_ac51-046287', 281.5105852, -2.5971097, 11.202, 11.215],
    ['2813m031_ac51-042316', 281.4783673, -2.6113444, 9.326, 9.314],
    ['2813m031_ac51-041999', 281.4723376, -2.6008704, 6.412, 6.62],
    ['2813m03

In [16]:
# pdacResultS = sorted( wisePDACQ['result']['table']['data'], key=lambda x: x[0] )

In [17]:
pdacResultD = { row[0]: row[1:] for row in wisePDACQ['result']['table']['data'] }

In [18]:
k = sorted(list(pdacResultD.keys()))[0]
print( k, pdacResultD[k])

2813m031_ac51-041856 [281.4933836, -2.6186303, 4.269, 1.777]


In [19]:
len(pdacResultD)

48

Same query at IPAC.  Note that we have to revert the change made in the PDAC version of the table, where the SQL reserved word `DEC` was replaced by `decl` as the column name for declination.

In [20]:
ipacQuery = wiseSelect.replace('decl','dec') + ' FROM allwise_p3as_psd ' + adqlWhere
ipacQuery = ipacQuery.replace(' ','+')
ipacQuery

"SELECT+source_id,+ra,+dec,+w1mpro,+w2mpro+FROM+allwise_p3as_psd+WHERE+CONTAINS(POINT('J2000',ra,dec),CIRCLE('J2000',281.5,-2.6,0.027777777777777776))=1"

In [21]:
wiseIPACQ = doIPACQuery(ipacQuery)

PDAC: Executing "SELECT+source_id,+ra,+dec,+w1mpro,+w2mpro+FROM+allwise_p3as_psd+WHERE+CONTAINS(POINT('J2000',ra,dec),CIRCLE('J2000',281.5,-2.6,0.027777777777777776))=1"
Before query (wall,cpu): 1528229879.250043 2.050661
After  query (wall,cpu,elapsed): 1528229880.6839502 2.088605 1.4339072704315186
IPAC query status code: 200


In [22]:
wiseIPACroot = ET.fromstring(wiseIPACQ)

In [23]:
wiseIPACroot.tag

'{http://www.ivoa.net/xml/VOTable/v1.3}VOTABLE'

We iterate through the header.  Note that the IRSA TAP service doesn't return UCDs for any but the positional columns.  (UCDs are still being developed for these tables as of early 2018.)

In [24]:
for child in wiseIPACroot[0][1]:
    print(child.tag, child.attrib)

{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'source_id', 'datatype': 'char', 'arraysize': '*'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'ra', 'datatype': 'double', 'precision': '7', 'ucd': 'pos.eq.ra;meta.main', 'unit': 'deg', 'width': '11'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'dec', 'datatype': 'double', 'precision': '7', 'ucd': 'pos.eq.dec;meta.main', 'unit': 'deg', 'width': '11'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'w1mpro', 'datatype': 'double', 'precision': '3', 'unit': 'mag', 'width': '7'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'w2mpro', 'datatype': 'double', 'precision': '3', 'unit': 'mag', 'width': '7'}
{http://www.ivoa.net/xml/VOTable/v1.3}DATA {}


In [25]:
ipacResultD = { row[0].text: [ float(row[1].text), 
   float(row[2].text), float(row[3].text), float(row[4].text) ] \
              for row in wiseIPACroot.iter('{http://www.ivoa.net/xml/VOTable/v1.3}TR') }

In [26]:
k = sorted(list(ipacResultD.keys()))[0]
print( k, ipacResultD[k])

2813m031_ac51-041856 [281.4933836, -2.6186303, 4.269, 1.7770000000000001]


In [27]:
len(ipacResultD)

48

In [28]:
pdacIDs = [ row for row in pdacResultD ]

In [29]:
ipacIDs = [ row for row in ipacResultD ]

See whether the same keys (IDs) were found in each table.

In [30]:
sorted(ipacIDs)==sorted(pdacIDs)

True

In [31]:
bothIDs = sorted([ row for row in pdacResultD if row in ipacResultD ])

In [32]:
len(bothIDs)

48

In [33]:
coaddObjectTestID = bothIDs[0] # pick this as the object ID to use for ID searches later
coaddObjectTestID

'2813m031_ac51-041856'

For IDs in both sets (all of them, we hope), compare the numerical results for the coordinate and magnitudes.

In [34]:
diffArray = [ [ ( pdacResultD[id][i] - ipacResultD[id][i] ) for id in bothIDs ] for i in list(range(4)) ]

Look for the largest deviations.

In [35]:
[ ( min(diffs), max(diffs) ) for diffs in diffArray ]

[(0.0, 5.684341886080802e-14),
 (-4.440892098500626e-16, 4.440892098500626e-16),
 (-1.7763568394002505e-15, 0.0),
 (-1.7763568394002505e-15, 0.0)]

#### AllWISE MultiEpoch Photometry Catalog (MEP), i.e., "ForcedSource"

First, do the same cone searches on the much larger ForcedSource-like table, the MultiEpoch Photometry Catalog or MEP.

`source_id_mf` is the ID of the corresponding coadded source.  We also add the time, since we are effectively querying a light curve here.

In [36]:
wiseSelect = 'SELECT source_id_mf, ra, decl, mjd, w1mpro_ep, w2mpro_ep'

In [37]:
wisePDACQ = doPDACQuery( wiseSelect + ' FROM wise_00.allwise_p3as_mep ' 
                        + qservWhere )

PDAC: Executing "SELECT source_id_mf, ra, decl, mjd, w1mpro_ep, w2mpro_ep FROM wise_00.allwise_p3as_mep WHERE qserv_areaspec_circle( 281.5, -2.6, 0.027777777777777776 )"
Before query (wall,cpu): 1528229880.8795462 2.300661
After  query (wall,cpu,elapsed): 1528229881.655211 2.318797 0.7756648063659668
PDAC query status code: 200


In [38]:
wisePDACQ

{'result': {'table': {'metadata': {'elements': [{'name': 'source_id_mf',
      'datatype': 'text'},
     {'name': 'ra', 'datatype': 'double'},
     {'name': 'decl', 'datatype': 'double'},
     {'name': 'mjd', 'datatype': 'double'},
     {'name': 'w1mpro_ep', 'datatype': 'double'},
     {'name': 'w2mpro_ep', 'datatype': 'double'}]},
   'data': [['2813m031_ac51-045492',
     281.5132724,
     -2.5922429,
     55287.3126044,
     10.949,
     10.496],
    ['2813m031_ac51-045492',
     281.5132724,
     -2.5922429,
     55469.8551685,
     11.149,
     10.74],
    ['2813m031_ac51-045492',
     281.5132724,
     -2.5922429,
     55469.9213842,
     11.072,
     10.772],
    ['2813m031_ac51-045492',
     281.5132724,
     -2.5922429,
     55287.3788201,
     10.91,
     10.499],
    ['2813m031_ac51-045492',
     281.5132724,
     -2.5922429,
     55287.4449084,
     10.815,
     10.536],
    ['2813m031_ac51-045492',
     281.5132724,
     -2.5922429,
     55287.5111241,
     10.944,
     10.

In [39]:
len(wisePDACQ['result']['table']['data'])

1377

In [40]:
# pdacResultS = sorted( wisePDACQ['result']['table']['data'], key=lambda x: x[0] )

Note that there can be more sources found in this search than in the coadded source catalog search above, because there are MEP entries for the Reject Table as well.

In [41]:
pdacObjectKeys = { row[0] for row in wisePDACQ['result']['table']['data'] }

In [42]:
len(pdacObjectKeys)

56

Make sure that the test object from the list above was found in this cone search.

In [43]:
coaddObjectTestID in pdacObjectKeys

True

Create a light curve table

In [44]:
pdacLightCurves = { source: {} for source in pdacObjectKeys }

In [45]:
for row in wisePDACQ['result']['table']['data']:
    pdacLightCurves[row[0]][row[3]] = [row[1], row[2], row[4], row[5]]

In [46]:
for source in sorted(list(pdacObjectKeys)):
    print( source, len(pdacLightCurves[source]) )

2813m031_ac51-041856 25
2813m031_ac51-041949 25
2813m031_ac51-041969 25
2813m031_ac51-041999 25
2813m031_ac51-042175 26
2813m031_ac51-042209 25
2813m031_ac51-042231 25
2813m031_ac51-042263 24
2813m031_ac51-042290 24
2813m031_ac51-042316 24
2813m031_ac51-042425 24
2813m031_ac51-042437 24
2813m031_ac51-042610 25
2813m031_ac51-042632 24
2813m031_ac51-042682 25
2813m031_ac51-042862 24
2813m031_ac51-042874 24
2813m031_ac51-042882 24
2813m031_ac51-042902 24
2813m031_ac51-042944 24
2813m031_ac51-042957 25
2813m031_ac51-043072 24
2813m031_ac51-043222 24
2813m031_ac51-043261 24
2813m031_ac51-043277 25
2813m031_ac51-043307 25
2813m031_ac51-043376 26
2813m031_ac51-043413 24
2813m031_ac51-043748 25
2813m031_ac51-043751 24
2813m031_ac51-043845 24
2813m031_ac51-043846 24
2813m031_ac51-043997 26
2813m031_ac51-044234 25
2813m031_ac51-044399 25
2813m031_ac51-044452 25
2813m031_ac51-044554 25
2813m031_ac51-045039 24
2813m031_ac51-045118 24
2813m031_ac51-045139 25
2813m031_ac51-045140 25
2813m031_ac51-04

In [47]:
pdacLightCurves[coaddObjectTestID]

{55292.4729725: [281.4933836, -2.6186303, 4.182, None],
 55287.9080369: [281.4933836, -2.6186303, 4.262, 2.575],
 55287.7757328: [281.4933836, -2.6186303, 4.334, 2.629],
 55287.6434288: [281.4933836, -2.6186303, 4.046, 2.51],
 55287.5773398: [281.4933836, -2.6186303, 4.254, 2.379],
 55287.577213: [281.4933836, -2.6186303, 4.208, 2.383],
 55287.5111241: [281.4933836, -2.6186303, 4.262, 2.661],
 55287.3126044: [281.4933836, -2.6186303, 4.241, 2.63],
 55287.1803003: [281.4933836, -2.6186303, 4.384, 2.575],
 55287.0479963: [281.4933836, -2.6186303, 4.262, 2.575],
 55286.9156922: [281.4933836, -2.6186303, 4.283, 2.52],
 55287.4449084: [281.4933836, -2.6186303, 4.339, 2.599],
 55287.3788201: [281.4933836, -2.6186303, 4.255, 2.235],
 55469.8551685: [281.4933836, -2.6186303, 4.555, 1.71],
 55469.3922967: [281.4933836, -2.6186303, 4.411, 2.607],
 55469.5245995: [281.4933836, -2.6186303, 4.307, 2.443],
 55469.5906878: [281.4933836, -2.6186303, 4.031, 2.636],
 55469.6567762: [281.4933836, -2.6186

Perform the same query at the IPAC TAP service.

In [48]:
ipacQuery = wiseSelect.replace('decl','dec') + ' FROM allwise_p3as_mep ' + adqlWhere
ipacQuery = ipacQuery.replace(' ','+')
ipacQuery

"SELECT+source_id_mf,+ra,+dec,+mjd,+w1mpro_ep,+w2mpro_ep+FROM+allwise_p3as_mep+WHERE+CONTAINS(POINT('J2000',ra,dec),CIRCLE('J2000',281.5,-2.6,0.027777777777777776))=1"

In [49]:
wiseIPACQ = doIPACQuery( ipacQuery )

PDAC: Executing "SELECT+source_id_mf,+ra,+dec,+mjd,+w1mpro_ep,+w2mpro_ep+FROM+allwise_p3as_mep+WHERE+CONTAINS(POINT('J2000',ra,dec),CIRCLE('J2000',281.5,-2.6,0.027777777777777776))=1"
Before query (wall,cpu): 1528229881.978902 2.6509840000000002
After  query (wall,cpu,elapsed): 1528229887.259392 2.7210769999999997 5.280489921569824
IPAC query status code: 200


In [50]:
wiseIPACroot = ET.fromstring(wiseIPACQ)

In [51]:
wiseIPACroot.tag

'{http://www.ivoa.net/xml/VOTable/v1.3}VOTABLE'

In [52]:
for child in wiseIPACroot[0][1]:
    print(child.tag, child.attrib)

{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'source_id_mf', 'datatype': 'char', 'arraysize': '*'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'ra', 'datatype': 'double', 'precision': '7', 'ucd': 'pos.eq.ra;meta.main', 'unit': 'deg', 'width': '11'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'dec', 'datatype': 'double', 'precision': '7', 'ucd': 'pos.eq.dec;meta.main', 'unit': 'deg', 'width': '11'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'mjd', 'datatype': 'double', 'precision': '8', 'unit': 'day', 'width': '14'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'w1mpro_ep', 'datatype': 'double', 'precision': '3', 'unit': 'mag', 'width': '7'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'w2mpro_ep', 'datatype': 'double', 'precision': '3', 'unit': 'mag', 'width': '7'}
{http://www.ivoa.net/xml/VOTable/v1.3}DATA {}


In [53]:
n=0
ipacObjectKeys = set()
for row in wiseIPACroot.iter('{http://www.ivoa.net/xml/VOTable/v1.3}TR'):
    n += 1
    ipacObjectKeys.add( row[0].text )
n

1377

In [54]:
len(ipacObjectKeys)

56

Make sure the same keys were found.

In [55]:
ipacObjectKeys == pdacObjectKeys

True

In [56]:
ipacLightCurves = { source: {} for source in ipacObjectKeys }

In [57]:
def floatOrNone( s ):
    return None if s==None else float(s)

In [58]:
for row in wiseIPACroot.iter('{http://www.ivoa.net/xml/VOTable/v1.3}TR'):
    ipacLightCurves[row[0].text][float(row[3].text)] = [float(row[1].text), float(row[2].text), floatOrNone(row[4].text), floatOrNone(row[5].text)]

In [59]:
for source in sorted(list(ipacObjectKeys)):
    print( source, len(ipacLightCurves[source]) )

2813m031_ac51-041856 25
2813m031_ac51-041949 25
2813m031_ac51-041969 25
2813m031_ac51-041999 25
2813m031_ac51-042175 26
2813m031_ac51-042209 25
2813m031_ac51-042231 25
2813m031_ac51-042263 24
2813m031_ac51-042290 24
2813m031_ac51-042316 24
2813m031_ac51-042425 24
2813m031_ac51-042437 24
2813m031_ac51-042610 25
2813m031_ac51-042632 24
2813m031_ac51-042682 25
2813m031_ac51-042862 24
2813m031_ac51-042874 24
2813m031_ac51-042882 24
2813m031_ac51-042902 24
2813m031_ac51-042944 24
2813m031_ac51-042957 25
2813m031_ac51-043072 24
2813m031_ac51-043222 24
2813m031_ac51-043261 24
2813m031_ac51-043277 25
2813m031_ac51-043307 25
2813m031_ac51-043376 26
2813m031_ac51-043413 24
2813m031_ac51-043748 25
2813m031_ac51-043751 24
2813m031_ac51-043845 24
2813m031_ac51-043846 24
2813m031_ac51-043997 26
2813m031_ac51-044234 25
2813m031_ac51-044399 25
2813m031_ac51-044452 25
2813m031_ac51-044554 25
2813m031_ac51-045039 24
2813m031_ac51-045118 24
2813m031_ac51-045139 25
2813m031_ac51-045140 25
2813m031_ac51-04

In [60]:
ipacLightCurves[coaddObjectTestID]

{55287.1803003: [281.4933836, -2.6186303, 4.384, 2.575],
 55287.0479963: [281.4933836, -2.6186303, 4.2620000000000005, 2.575],
 55287.7757328: [281.4933836, -2.6186303, 4.3340000000000005, 2.629],
 55287.9080369: [281.4933836, -2.6186303, 4.2620000000000005, 2.575],
 55469.7229919: [281.4933836, -2.6186303, 4.316, 1.5050000000000001],
 55469.7890802: [281.4933836, -2.6186303, 4.2, 2.427],
 55287.3126044: [281.4933836, -2.6186303, 4.2410000000000005, 2.63],
 55287.5773398: [281.4933836, -2.6186303, 4.2540000000000004, 2.379],
 55287.6434288: [281.4933836, -2.6186303, 4.046, 2.5100000000000002],
 55469.5906878: [281.4933836, -2.6186303, 4.031000000000001, 2.636],
 55469.6567762: [281.4933836, -2.6186303, 4.423, 2.197],
 55470.2520805: [281.4933836, -2.6186303, 4.393, 2.129],
 55286.9156922: [281.4933836, -2.6186303, 4.283, 2.52],
 55287.5111241: [281.4933836, -2.6186303, 4.2620000000000005, 2.661],
 55287.577213: [281.4933836, -2.6186303, 4.208, 2.383],
 55469.3922967: [281.4933836, -2.6

Compare the epochs of the two light curves.

In [61]:
pdacTimes = np.array( sorted([ t for t in pdacLightCurves[coaddObjectTestID] ]) )

In [62]:
ipacTimes = np.array( sorted([ t for t in ipacLightCurves[coaddObjectTestID] ]) )

In [63]:
ipacTimes-pdacTimes

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

Ensure that the coordinates are identical in every epoch in both cases.  Just look at the one test object for now.

In [64]:
pdacCoords = { (pdacLightCurves[coaddObjectTestID][row][0],pdacLightCurves[coaddObjectTestID][row][1]) for row in pdacLightCurves[coaddObjectTestID] }

In [65]:
pdacCoords

{(281.4933836, -2.6186303)}

In [66]:
ipacCoords = { (ipacLightCurves[coaddObjectTestID][row][0],ipacLightCurves[coaddObjectTestID][row][1]) for row in ipacLightCurves[coaddObjectTestID] }

In [67]:
ipacCoords

{(281.4933836, -2.6186303)}

In [68]:
pdacCoords == ipacCoords

True

#### Single-epoch source table example: 4-band photometry table (analogous to LSST Source)

(Note that we do not use the NEOWISE Year 1 table for this because the comparison with the IPAC database is not exact: in IPAC the NEOWISE database contains all the data from the reactivated mission; it's not segregated by year.  Therefore the corresponding IPAC table is now more than four times larger than the one in PDAC.)

In [69]:
wiseSelect = 'SELECT source_id, ra, decl, mjd, w1mpro, w2mpro'

In [70]:
wisePDACQ = doPDACQuery( wiseSelect + ' FROM wise_4band_00.allsky_4band_p1bs_psd '
                        + qservWhere )

PDAC: Executing "SELECT source_id, ra, decl, mjd, w1mpro, w2mpro FROM wise_4band_00.allsky_4band_p1bs_psd WHERE qserv_areaspec_circle( 281.5, -2.6, 0.027777777777777776 )"
Before query (wall,cpu): 1528229887.5890749 3.053938
After  query (wall,cpu,elapsed): 1528229888.0637162 3.0753489999999997 0.4746413230895996
PDAC query status code: 200


In [71]:
wisePDACQ

{'result': {'table': {'metadata': {'elements': [{'name': 'source_id',
      'datatype': 'text'},
     {'name': 'ra', 'datatype': 'double'},
     {'name': 'decl', 'datatype': 'double'},
     {'name': 'mjd', 'datatype': 'double'},
     {'name': 'w1mpro', 'datatype': 'double'},
     {'name': 'w2mpro', 'datatype': 'double'}]},
   'data': [['03253b121-011980',
     281.5073016,
     -2.578184,
     55287.1803,
     11.938,
     11.557],
    ['03260a096-011061', 281.5163333, -2.6088027, 55287.37882, 12.12, 10.632],
    ['03253b121-005671', 281.4860309, -2.605907, 55287.1803, 10.644, 10.579],
    ['03257b121-005182', 281.476468, -2.5947561, 55287.312604, 10.707, 11.042],
    ['03249b121-003380', 281.4891147, -2.6102491, 55287.047996, 9.758, 9.752],
    ['03249b121-003981', 281.482014, -2.5870217, 55287.047996, 9.956, 9.954],
    ['03276a096-009041', 281.4819597, -2.6006732, 55287.908037, 9.133, 9.141],
    ['03276a096-001178', 281.5205629, -2.5822778, 55287.908037, 8.616, 8.284],
    ['03413b

In [72]:
len(wisePDACQ['result']['table']['data'])

506

This number should be less than for the MEP search above, because it includes only sources detectable in a single epoch.  All the keys should be unique - no association analysis was performed in the WISE production.

In [73]:
pdacSourceKeys = { row[0] for row in wisePDACQ['result']['table']['data'] }

In [74]:
len(pdacSourceKeys)

506

In [75]:
pdacSourceResultD = { row[0]: row[1:] for row in wisePDACQ['result']['table']['data'] }

Repeat for the IPAC version of the table.

In [76]:
ipacQuery = wiseSelect.replace('decl','dec') + ' FROM allsky_4band_p1bs_psd ' + adqlWhere
ipacQuery = ipacQuery.replace(' ','+')
ipacQuery

"SELECT+source_id,+ra,+dec,+mjd,+w1mpro,+w2mpro+FROM+allsky_4band_p1bs_psd+WHERE+CONTAINS(POINT('J2000',ra,dec),CIRCLE('J2000',281.5,-2.6,0.027777777777777776))=1"

In [77]:
wiseIPACQ = doIPACQuery( ipacQuery )

PDAC: Executing "SELECT+source_id,+ra,+dec,+mjd,+w1mpro,+w2mpro+FROM+allsky_4band_p1bs_psd+WHERE+CONTAINS(POINT('J2000',ra,dec),CIRCLE('J2000',281.5,-2.6,0.027777777777777776))=1"
Before query (wall,cpu): 1528229888.2351491 3.2573429999999997
After  query (wall,cpu,elapsed): 1528229889.71124 3.2948399999999998 1.476090908050537
IPAC query status code: 200


In [78]:
wiseIPACroot = ET.fromstring(wiseIPACQ)

In [79]:
wiseIPACroot.tag

'{http://www.ivoa.net/xml/VOTable/v1.3}VOTABLE'

In [80]:
for child in wiseIPACroot[0][1]:
    print(child.tag, child.attrib)

{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'source_id', 'datatype': 'char', 'arraysize': '*'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'ra', 'datatype': 'double', 'precision': '7', 'ucd': 'pos.eq.ra;meta.main', 'unit': 'deg', 'width': '11'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'dec', 'datatype': 'double', 'precision': '7', 'ucd': 'pos.eq.dec;meta.main', 'unit': 'deg', 'width': '11'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'mjd', 'datatype': 'double', 'precision': '6', 'unit': 'mjdate', 'width': '12'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'w1mpro', 'datatype': 'double', 'precision': '3', 'unit': 'mag', 'width': '7'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'w2mpro', 'datatype': 'double', 'precision': '3', 'unit': 'mag', 'width': '7'}
{http://www.ivoa.net/xml/VOTable/v1.3}DATA {}


In [81]:
n=0
ipacSourceKeys = set()
for row in wiseIPACroot.iter('{http://www.ivoa.net/xml/VOTable/v1.3}TR'):
    n += 1
    ipacSourceKeys.add( row[0].text )
n

506

In [82]:
len(ipacSourceKeys)

506

Make sure the same keys were found.

In [83]:
ipacSourceKeys == pdacSourceKeys

True

In [84]:
ipacSourceResultD = { row[0].text: [ float(row[1].text), 
   float(row[2].text), float(row[3].text), floatOrNone(row[4].text), floatOrNone(row[5].text) ] \
              for row in wiseIPACroot.iter('{http://www.ivoa.net/xml/VOTable/v1.3}TR') }

In [85]:
bothSourceKeys = sorted([ k for k in pdacSourceKeys if k in ipacSourceKeys ])

In [86]:
len(bothSourceKeys)

506

Save one of the IDs for use in the search-by-ID section below.

In [87]:
singleTestSourceId = bothSourceKeys[0]
singleTestSourceId

'03245b121-000010'

For IDs in both sets (all of them, we hope), compare the numerical results for the coordinate and magnitudes.

In [88]:
def noneSubtract( a, b ):
    if (a == None and b == None): print( 'both none' )
    return 0 if (a == None and b == None) else ( None if (a == None or b == None) else ( a-b ) )

In [89]:
diffArray = [ [ noneSubtract( pdacSourceResultD[id][i], ipacSourceResultD[id][i] ) for id in bothSourceKeys ] for i in list(range(5)) ]

both none
both none
both none
both none
both none


Look for the largest deviations.

In [90]:
[ ( min(diffs), max(diffs) ) for diffs in diffArray ]

[(-5.684341886080802e-14, 5.684341886080802e-14),
 (-4.440892098500626e-16, 4.440892098500626e-16),
 (0.0, 0.0),
 (-1.7763568394002505e-15, 0.0),
 (-1.7763568394002505e-15, 0.0)]

### Object-by-name searches

#### Object-like coadded source catalog

In [91]:
wiseSelect = 'SELECT source_id, cntr, ra, decl, w1mpro, w2mpro'

In [92]:
testIDWhere = 'WHERE source_id=\'' + coaddObjectTestID + '\''

In [93]:
wisePDACQ = doPDACQuery( wiseSelect + ' FROM wise_00.allwise_p3as_psd ' 
                        + testIDWhere )

PDAC: Executing "SELECT source_id, cntr, ra, decl, w1mpro, w2mpro FROM wise_00.allwise_p3as_psd WHERE source_id='2813m031_ac51-041856'"
Before query (wall,cpu): 1528229889.947605 3.5553679999999996
After  query (wall,cpu,elapsed): 1528229890.128536 3.567798 0.18093109130859375
PDAC query status code: 200


In [94]:
wisePDACQ

{'result': {'table': {'metadata': {'elements': [{'name': 'source_id',
      'datatype': 'text'},
     {'name': 'cntr', 'datatype': 'long'},
     {'name': 'ra', 'datatype': 'double'},
     {'name': 'decl', 'datatype': 'double'},
     {'name': 'w1mpro', 'datatype': 'double'},
     {'name': 'w2mpro', 'datatype': 'double'}]},
   'data': [['2813m031_ac51-041856',
     2813003101351041856,
     281.4933836,
     -2.6186303,
     4.269,
     1.777]]}}}

In [95]:
wisePDACQ['result']['table']['data']

[['2813m031_ac51-041856',
  2813003101351041856,
  281.4933836,
  -2.6186303,
  4.269,
  1.777]]

In [96]:
wisePDACQ['result']['table']['data'][0][0]==coaddObjectTestID

True

Save the numeric ID (`cntr`, which has a 1:1 relationship with `source_id`) for future use, and save the RA and DEC for use in a cone search.

In [97]:
testNumericID = wisePDACQ['result']['table']['data'][0][1]
testNumericID

2813003101351041856

In [98]:
testObjectRa  = wisePDACQ['result']['table']['data'][0][2]
testObjectDec = wisePDACQ['result']['table']['data'][0][3]
( testObjectRa, testObjectDec )

(281.4933836, -2.6186303)

Repeat the query on the IRSA services.

In [99]:
ipacQuery = wiseSelect.replace('decl','dec') + ' FROM allwise_p3as_psd ' + testIDWhere
ipacQuery = ipacQuery.replace(' ','+')
ipacQuery

"SELECT+source_id,+cntr,+ra,+dec,+w1mpro,+w2mpro+FROM+allwise_p3as_psd+WHERE+source_id='2813m031_ac51-041856'"

In [100]:
wiseIPACQ = doIPACQuery( ipacQuery )

PDAC: Executing "SELECT+source_id,+cntr,+ra,+dec,+w1mpro,+w2mpro+FROM+allwise_p3as_psd+WHERE+source_id='2813m031_ac51-041856'"
Before query (wall,cpu): 1528229890.211793 3.661985
After  query (wall,cpu,elapsed): 1528229892.972637 3.696136 2.7608439922332764
IPAC query status code: 200


In [101]:
wiseIPACroot = ET.fromstring(wiseIPACQ)

In [102]:
wiseIPACroot.tag

'{http://www.ivoa.net/xml/VOTable/v1.3}VOTABLE'

In [103]:
for child in wiseIPACroot[0][1]:
    print(child.tag, child.attrib)

{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'source_id', 'datatype': 'char', 'arraysize': '*'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'cntr', 'datatype': 'long', 'ucd': 'meta.record;meta.main'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'ra', 'datatype': 'double', 'precision': '7', 'ucd': 'pos.eq.ra;meta.main', 'unit': 'deg', 'width': '11'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'dec', 'datatype': 'double', 'precision': '7', 'ucd': 'pos.eq.dec;meta.main', 'unit': 'deg', 'width': '11'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'w1mpro', 'datatype': 'double', 'precision': '3', 'unit': 'mag', 'width': '7'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'w2mpro', 'datatype': 'double', 'precision': '3', 'unit': 'mag', 'width': '7'}
{http://www.ivoa.net/xml/VOTable/v1.3}DATA {}


In [104]:
ipacResultD = { row[0].text: [ float(row[1].text), 
   float(row[2].text), float(row[3].text), float(row[4].text), float(row[5].text) ] \
              for row in wiseIPACroot.iter('{http://www.ivoa.net/xml/VOTable/v1.3}TR') }

In [105]:
ipacResultD

{'2813m031_ac51-041856': [2.813003101351042e+18,
  281.4933836,
  -2.6186303,
  4.269,
  1.7770000000000001]}

#### Use the ID to search for multiple entries in the ForcedSource-like table

In [106]:
wiseSelect = 'SELECT cntr, source_id_mf, cntr_mf, mjd, ra, decl, w1mpro_ep, w2mpro_ep'

In [107]:
testIDWhere = 'WHERE source_id_mf=\'' + coaddObjectTestID + '\''

In [108]:
wisePDACQ = doPDACQuery( wiseSelect + ' FROM wise_00.allwise_p3as_mep ' 
                        + testIDWhere )

PDAC: Executing "SELECT cntr, source_id_mf, cntr_mf, mjd, ra, decl, w1mpro_ep, w2mpro_ep FROM wise_00.allwise_p3as_mep WHERE source_id_mf='2813m031_ac51-041856'"
Before query (wall,cpu): 1528229893.061141 3.7947770000000003
After  query (wall,cpu,elapsed): 1528229893.254595 3.808583 0.19345402717590332
PDAC query status code: 200


In [109]:
wisePDACQ

{'result': {'table': {'metadata': {'elements': [{'name': 'cntr',
      'datatype': 'long'},
     {'name': 'source_id_mf', 'datatype': 'text'},
     {'name': 'cntr_mf', 'datatype': 'long'},
     {'name': 'mjd', 'datatype': 'double'},
     {'name': 'ra', 'datatype': 'double'},
     {'name': 'decl', 'datatype': 'double'},
     {'name': 'w1mpro_ep', 'datatype': 'double'},
     {'name': 'w2mpro_ep', 'datatype': 'double'}]},
   'data': [[2813003110418560013,
     '2813m031_ac51-041856',
     2813003101351041856,
     55292.4729725,
     281.4933836,
     -2.6186303,
     4.182,
     None],
    [2813003110418560012,
     '2813m031_ac51-041856',
     2813003101351041856,
     55287.9080369,
     281.4933836,
     -2.6186303,
     4.262,
     2.575],
    [2813003110418560011,
     '2813m031_ac51-041856',
     2813003101351041856,
     55287.7757328,
     281.4933836,
     -2.6186303,
     4.334,
     2.629],
    [2813003110418560010,
     '2813m031_ac51-041856',
     2813003101351041856,
     5

Make sure that the things that should be constant - properties of the object, not the individual observations - are constant.

In [110]:
pdacFSTableConstsStr = { (row[1], row[2], row[4], row[5]) for row in wisePDACQ['result']['table']['data'] }
pdacFSTableConstsStr

{('2813m031_ac51-041856', 2813003101351041856, 281.4933836, -2.6186303)}

In [111]:
len(pdacFSTableConstsStr)==1

True

Extract the time-dependent components.

In [112]:
pdacFSTableStr = { row[3]: [row[0], row[6], row[7]] for row in wisePDACQ['result']['table']['data'] }
pdacFSTableStr

{55292.4729725: [2813003110418560013, 4.182, None],
 55287.9080369: [2813003110418560012, 4.262, 2.575],
 55287.7757328: [2813003110418560011, 4.334, 2.629],
 55287.6434288: [2813003110418560010, 4.046, 2.51],
 55287.5773398: [2813003110418560009, 4.254, 2.379],
 55287.577213: [2813003110418560008, 4.208, 2.383],
 55287.5111241: [2813003110418560007, 4.262, 2.661],
 55287.3126044: [2813003110418560004, 4.241, 2.63],
 55287.1803003: [2813003110418560003, 4.384, 2.575],
 55287.0479963: [2813003110418560002, 4.262, 2.575],
 55286.9156922: [2813003110418560001, 4.283, 2.52],
 55287.4449084: [2813003110418560006, 4.339, 2.599],
 55287.3788201: [2813003110418560005, 4.255, 2.235],
 55469.8551685: [2813003110418560021, 4.555, 1.71],
 55469.3922967: [2813003110418560015, 4.411, 2.607],
 55469.5245995: [2813003110418560016, 4.307, 2.443],
 55469.5906878: [2813003110418560017, 4.031, 2.636],
 55469.6567762: [2813003110418560018, 4.423, 2.197],
 55469.7229919: [2813003110418560019, 4.316, 1.505],

Make sure this is the same light curve that we saw when processing the cone search above.  This checks indexing, partitioning, and other aspects of the table loading for self-consistency.

In [113]:
len(pdacFSTableStr)

25

In [114]:
len(pdacFSTableStr) == len(pdacLightCurves[coaddObjectTestID])

True

In [115]:
sorted(pdacFSTableStr.keys()) == sorted(pdacLightCurves[coaddObjectTestID].keys())

True

Repeat the search at IRSA

In [116]:
ipacQuery = wiseSelect.replace('decl','dec') + ' FROM allwise_p3as_mep ' + testIDWhere
ipacQuery = ipacQuery.replace(' ','+')
ipacQuery

"SELECT+cntr,+source_id_mf,+cntr_mf,+mjd,+ra,+dec,+w1mpro_ep,+w2mpro_ep+FROM+allwise_p3as_mep+WHERE+source_id_mf='2813m031_ac51-041856'"

In [117]:
wiseIPACQ = doIPACQuery( ipacQuery )

PDAC: Executing "SELECT+cntr,+source_id_mf,+cntr_mf,+mjd,+ra,+dec,+w1mpro_ep,+w2mpro_ep+FROM+allwise_p3as_mep+WHERE+source_id_mf='2813m031_ac51-041856'"
Before query (wall,cpu): 1528229893.38297 3.942755
After  query (wall,cpu,elapsed): 1528229896.0831869 3.971898 2.700216770172119
IPAC query status code: 200


In [118]:
wiseIPACroot = ET.fromstring(wiseIPACQ)

In [119]:
wiseIPACroot.tag

'{http://www.ivoa.net/xml/VOTable/v1.3}VOTABLE'

In [120]:
for child in wiseIPACroot[0][1]:
    print(child.tag, child.attrib)

{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'cntr', 'datatype': 'long', 'ucd': 'meta.record;meta.main'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'source_id_mf', 'datatype': 'char', 'arraysize': '*'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'cntr_mf', 'datatype': 'long'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'mjd', 'datatype': 'double', 'precision': '8', 'unit': 'day', 'width': '14'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'ra', 'datatype': 'double', 'precision': '7', 'ucd': 'pos.eq.ra;meta.main', 'unit': 'deg', 'width': '11'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'dec', 'datatype': 'double', 'precision': '7', 'ucd': 'pos.eq.dec;meta.main', 'unit': 'deg', 'width': '11'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'w1mpro_ep', 'datatype': 'double', 'precision': '3', 'unit': 'mag', 'width': '7'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'w2mpro_ep', 'datatype': 'double', 'precision': '3', 'unit

Check for the things that don't change.

In [121]:
ipacFSTableConstsStr = { (row[1].text, row[2].text, row[4].text, row[5].text) \
                        for row in wiseIPACroot.iter('{http://www.ivoa.net/xml/VOTable/v1.3}TR') }
ipacFSTableConstsStr

{('2813m031_ac51-041856',
  '2813003101351041856',
  '281.49338360000002',
  '-2.6186303')}

In [122]:
pdacFSTableConstsStr

{('2813m031_ac51-041856', 2813003101351041856, 281.4933836, -2.6186303)}

Get the light curve.

In [123]:
ipacFSTableStr = { float(row[3].text): [int(row[0].text), floatOrNone(row[6].text), floatOrNone(row[7].text)] \
                     for row in wiseIPACroot.iter('{http://www.ivoa.net/xml/VOTable/v1.3}TR') }
ipacFSTableStr

{55287.1803003: [2813003110418560003, 4.384, 2.575],
 55287.0479963: [2813003110418560002, 4.2620000000000005, 2.575],
 55287.3788201: [2813003110418560005, 4.255, 2.2350000000000003],
 55287.4449084: [2813003110418560006, 4.339, 2.599],
 55287.5111241: [2813003110418560007, 4.2620000000000005, 2.661],
 55287.577213: [2813003110418560008, 4.208, 2.383],
 55287.5773398: [2813003110418560009, 4.2540000000000004, 2.379],
 55287.6434288: [2813003110418560010, 4.046, 2.5100000000000002],
 55287.7757328: [2813003110418560011, 4.3340000000000005, 2.629],
 55287.9080369: [2813003110418560012, 4.2620000000000005, 2.575],
 55292.4729725: [2813003110418560013, 4.182, None],
 55469.2599927: [2813003110418560014, 4.352, 1.718],
 55469.3922967: [2813003110418560015, 4.4110000000000005, 2.607],
 55469.5245995: [2813003110418560016, 4.307, 2.443],
 55469.5906878: [2813003110418560017, 4.031000000000001, 2.636],
 55469.6567762: [2813003110418560018, 4.423, 2.197],
 55469.7229919: [2813003110418560019, 

Compare with the same light curve from the PDAC.  We only compare keys because we've already done a comparison of values between PDAC and IRSA above.

In [124]:
sorted(ipacFSTableStr.keys()) == sorted(pdacFSTableStr.keys())

True

#### Searches by numeric ID, performed solely for timing checks.

It was observed that the database configuration in the PDAC Qserv instance had not used the WISE object and source numeric IDs in shard-level indexes or in an instance-wide "secondary index" that allows Qserv to determine which shard(s) to search.  Only the string IDs (e.g., `source_id`) were so indexed.  This makes numeric index searches full table scans.

A workaround including a small spatial cone around the known location of an object was tested, mainly because this workaround was used in the initial Portal implementation, and to collect comparison data with IRSA.

In [125]:
wiseSelect = 'SELECT source_id, cntr, ra, decl, w1mpro, w2mpro'

In [126]:
testIDWhere = 'WHERE qserv_areaspec_circle(' \
+ repr(testObjectRa) + ',' + repr(testObjectDec) \
+ ',0.005) AND cntr=' + repr(testNumericID)
testIDWhere

'WHERE qserv_areaspec_circle(281.4933836,-2.6186303,0.005) AND cntr=2813003101351041856'

In [127]:
wisePDACQ = doPDACQuery( wiseSelect + ' FROM wise_00.allwise_p3as_psd ' 
                        + testIDWhere )

PDAC: Executing "SELECT source_id, cntr, ra, decl, w1mpro, w2mpro FROM wise_00.allwise_p3as_psd WHERE qserv_areaspec_circle(281.4933836,-2.6186303,0.005) AND cntr=2813003101351041856"
Before query (wall,cpu): 1528229896.210485 4.108271
After  query (wall,cpu,elapsed): 1528229896.3999949 4.120315 0.18950986862182617
PDAC query status code: 200


In [128]:
wisePDACQ['result']['table']['data']

[['2813m031_ac51-041856',
  2813003101351041856,
  281.4933836,
  -2.6186303,
  4.269,
  1.777]]

The equivalent query at IRSA does not require a spatial restriction.

In [129]:
ipacQuery = wiseSelect.replace('decl','dec') + ' FROM allwise_p3as_psd ' \
+ 'WHERE cntr=' + repr(testNumericID)
ipacQuery = ipacQuery.replace(' ','+')
ipacQuery

'SELECT+source_id,+cntr,+ra,+dec,+w1mpro,+w2mpro+FROM+allwise_p3as_psd+WHERE+cntr=2813003101351041856'

In [130]:
wiseIPACQ = doIPACQuery( ipacQuery )

PDAC: Executing "SELECT+source_id,+cntr,+ra,+dec,+w1mpro,+w2mpro+FROM+allwise_p3as_psd+WHERE+cntr=2813003101351041856"
Before query (wall,cpu): 1528229896.434091 4.159813
After  query (wall,cpu,elapsed): 1528229899.218554 4.1914679999999995 2.7844629287719727
IPAC query status code: 200


In [131]:
wiseIPACroot = ET.fromstring(wiseIPACQ)

In [132]:
wiseIPACroot.tag

'{http://www.ivoa.net/xml/VOTable/v1.3}VOTABLE'

In [133]:
for child in wiseIPACroot[0][1]:
    print(child.tag, child.attrib)

{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'source_id', 'datatype': 'char', 'arraysize': '*'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'cntr', 'datatype': 'long', 'ucd': 'meta.record;meta.main'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'ra', 'datatype': 'double', 'precision': '7', 'ucd': 'pos.eq.ra;meta.main', 'unit': 'deg', 'width': '11'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'dec', 'datatype': 'double', 'precision': '7', 'ucd': 'pos.eq.dec;meta.main', 'unit': 'deg', 'width': '11'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'w1mpro', 'datatype': 'double', 'precision': '3', 'unit': 'mag', 'width': '7'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'w2mpro', 'datatype': 'double', 'precision': '3', 'unit': 'mag', 'width': '7'}
{http://www.ivoa.net/xml/VOTable/v1.3}DATA {}


In [134]:
ipacResultD = { row[0].text: [ float(row[1].text), 
   float(row[2].text), float(row[3].text), float(row[4].text), float(row[5].text) ] \
              for row in wiseIPACroot.iter('{http://www.ivoa.net/xml/VOTable/v1.3}TR') }

In [135]:
ipacResultD

{'2813m031_ac51-041856': [2.813003101351042e+18,
  281.4933836,
  -2.6186303,
  4.269,
  1.7770000000000001]}

Now ForcedSource at PDAC:

In [136]:
wiseSelect = 'SELECT cntr, source_id_mf, cntr_mf, mjd, ra, decl, w1mpro_ep, w2mpro_ep'

In [137]:
testIDWhere = 'WHERE qserv_areaspec_circle(' \
+ repr(testObjectRa) + ',' + repr(testObjectDec) \
+ ',0.005) AND cntr_mf=' + repr(testNumericID)
testIDWhere

'WHERE qserv_areaspec_circle(281.4933836,-2.6186303,0.005) AND cntr_mf=2813003101351041856'

In [138]:
wisePDACQ = doPDACQuery( wiseSelect + ' FROM wise_00.allwise_p3as_mep ' 
                        + testIDWhere )

PDAC: Executing "SELECT cntr, source_id_mf, cntr_mf, mjd, ra, decl, w1mpro_ep, w2mpro_ep FROM wise_00.allwise_p3as_mep WHERE qserv_areaspec_circle(281.4933836,-2.6186303,0.005) AND cntr_mf=2813003101351041856"
Before query (wall,cpu): 1528229899.3069959 4.2899009999999995
After  query (wall,cpu,elapsed): 1528229899.727374 4.302849999999999 0.4203782081604004
PDAC query status code: 200


In [139]:
wisePDACQ['result']['table']['data']

[[2813003110418560013,
  '2813m031_ac51-041856',
  2813003101351041856,
  55292.4729725,
  281.4933836,
  -2.6186303,
  4.182,
  None],
 [2813003110418560012,
  '2813m031_ac51-041856',
  2813003101351041856,
  55287.9080369,
  281.4933836,
  -2.6186303,
  4.262,
  2.575],
 [2813003110418560011,
  '2813m031_ac51-041856',
  2813003101351041856,
  55287.7757328,
  281.4933836,
  -2.6186303,
  4.334,
  2.629],
 [2813003110418560010,
  '2813m031_ac51-041856',
  2813003101351041856,
  55287.6434288,
  281.4933836,
  -2.6186303,
  4.046,
  2.51],
 [2813003110418560009,
  '2813m031_ac51-041856',
  2813003101351041856,
  55287.5773398,
  281.4933836,
  -2.6186303,
  4.254,
  2.379],
 [2813003110418560008,
  '2813m031_ac51-041856',
  2813003101351041856,
  55287.577213,
  281.4933836,
  -2.6186303,
  4.208,
  2.383],
 [2813003110418560007,
  '2813m031_ac51-041856',
  2813003101351041856,
  55287.5111241,
  281.4933836,
  -2.6186303,
  4.262,
  2.661],
 [2813003110418560004,
  '2813m031_ac51-0418

In [140]:
len(wisePDACQ['result']['table']['data'])

25

The equivalent ForcedSource query at IRSA does not require a spatial restriction.

In [141]:
ipacQuery = wiseSelect.replace('decl','dec') + ' FROM allwise_p3as_mep ' \
+ 'WHERE cntr_mf=' + repr(testNumericID)
ipacQuery = ipacQuery.replace(' ','+')
ipacQuery

'SELECT+cntr,+source_id_mf,+cntr_mf,+mjd,+ra,+dec,+w1mpro_ep,+w2mpro_ep+FROM+allwise_p3as_mep+WHERE+cntr_mf=2813003101351041856'

In [142]:
wiseIPACQ = doIPACQuery( ipacQuery )

PDAC: Executing "SELECT+cntr,+source_id_mf,+cntr_mf,+mjd,+ra,+dec,+w1mpro_ep,+w2mpro_ep+FROM+allwise_p3as_mep+WHERE+cntr_mf=2813003101351041856"
Before query (wall,cpu): 1528229899.780811 4.362155
After  query (wall,cpu,elapsed): 1528229902.4956028 4.392689 2.714791774749756
IPAC query status code: 200


In [143]:
wiseIPACroot = ET.fromstring(wiseIPACQ)

In [144]:
wiseIPACroot.tag

'{http://www.ivoa.net/xml/VOTable/v1.3}VOTABLE'

In [145]:
for child in wiseIPACroot[0][1]:
    print(child.tag, child.attrib)

{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'cntr', 'datatype': 'long', 'ucd': 'meta.record;meta.main'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'source_id_mf', 'datatype': 'char', 'arraysize': '*'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'cntr_mf', 'datatype': 'long'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'mjd', 'datatype': 'double', 'precision': '8', 'unit': 'day', 'width': '14'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'ra', 'datatype': 'double', 'precision': '7', 'ucd': 'pos.eq.ra;meta.main', 'unit': 'deg', 'width': '11'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'dec', 'datatype': 'double', 'precision': '7', 'ucd': 'pos.eq.dec;meta.main', 'unit': 'deg', 'width': '11'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'w1mpro_ep', 'datatype': 'double', 'precision': '3', 'unit': 'mag', 'width': '7'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'w2mpro_ep', 'datatype': 'double', 'precision': '3', 'unit

#### Finally, Source-like queries by identifier

In [146]:
wiseSelect = 'SELECT source_id, ra, decl, mjd, w1mpro, w2mpro'

In [147]:
testIDWhere = 'WHERE source_id=\'' + singleTestSourceId + '\''

In [148]:
wisePDACQ = doPDACQuery( wiseSelect + ' FROM wise_4band_00.allsky_4band_p1bs_psd ' 
                        + testIDWhere )

PDAC: Executing "SELECT source_id, ra, decl, mjd, w1mpro, w2mpro FROM wise_4band_00.allsky_4band_p1bs_psd WHERE source_id='03245b121-000010'"
Before query (wall,cpu): 1528229902.559244 4.463578
After  query (wall,cpu,elapsed): 1528229902.737783 4.4779409999999995 0.17853903770446777
PDAC query status code: 200


In [149]:
wisePDACQ

{'result': {'table': {'metadata': {'elements': [{'name': 'source_id',
      'datatype': 'text'},
     {'name': 'ra', 'datatype': 'double'},
     {'name': 'decl', 'datatype': 'double'},
     {'name': 'mjd', 'datatype': 'double'},
     {'name': 'w1mpro', 'datatype': 'double'},
     {'name': 'w2mpro', 'datatype': 'double'}]},
   'data': [['03245b121-000010',
     281.4933339,
     -2.6188357,
     55286.915692,
     4.359,
     2.212]]}}}

In [150]:
wisePDACQ['result']['table']['data']

[['03245b121-000010', 281.4933339, -2.6188357, 55286.915692, 4.359, 2.212]]

In [151]:
wisePDACQ['result']['table']['data'][0][0]==singleTestSourceId

True

Repeat the query on the IRSA services.

In [152]:
ipacQuery = wiseSelect.replace('decl','dec') + ' FROM allsky_4band_p1bs_psd ' + testIDWhere
ipacQuery = ipacQuery.replace(' ','+')
ipacQuery

"SELECT+source_id,+ra,+dec,+mjd,+w1mpro,+w2mpro+FROM+allsky_4band_p1bs_psd+WHERE+source_id='03245b121-000010'"

In [153]:
wiseIPACQ = doIPACQuery( ipacQuery )

PDAC: Executing "SELECT+source_id,+ra,+dec,+mjd,+w1mpro,+w2mpro+FROM+allsky_4band_p1bs_psd+WHERE+source_id='03245b121-000010'"
Before query (wall,cpu): 1528229902.8105862 4.54557
After  query (wall,cpu,elapsed): 1528229904.041273 4.577364 1.230686902999878
IPAC query status code: 200


In [154]:
wiseIPACroot = ET.fromstring(wiseIPACQ)

In [155]:
wiseIPACroot.tag

'{http://www.ivoa.net/xml/VOTable/v1.3}VOTABLE'

In [156]:
for child in wiseIPACroot[0][1]:
    print(child.tag, child.attrib)

{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'source_id', 'datatype': 'char', 'arraysize': '*'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'ra', 'datatype': 'double', 'precision': '7', 'ucd': 'pos.eq.ra;meta.main', 'unit': 'deg', 'width': '11'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'dec', 'datatype': 'double', 'precision': '7', 'ucd': 'pos.eq.dec;meta.main', 'unit': 'deg', 'width': '11'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'mjd', 'datatype': 'double', 'precision': '6', 'unit': 'mjdate', 'width': '12'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'w1mpro', 'datatype': 'double', 'precision': '3', 'unit': 'mag', 'width': '7'}
{http://www.ivoa.net/xml/VOTable/v1.3}FIELD {'name': 'w2mpro', 'datatype': 'double', 'precision': '3', 'unit': 'mag', 'width': '7'}
{http://www.ivoa.net/xml/VOTable/v1.3}DATA {}


In [157]:
ipacResultD = { row[0].text: [ float(row[1].text), 
   float(row[2].text), float(row[3].text), float(row[4].text), float(row[5].text) ] \
              for row in wiseIPACroot.iter('{http://www.ivoa.net/xml/VOTable/v1.3}TR') }

In [158]:
ipacResultD

{'03245b121-000010': [281.4933339, -2.6188357, 55286.915692, 4.359, 2.212]}